In [1]:
import numpy as np
import os
import dask.array as da
import glob,os
from tqdm import tqdm
import matplotlib.pylab as plt
def read_im(fl,return_pos=False,ncols=4):
    data = os.path.dirname(fl)+os.sep+os.path.basename(fl).split('_')[-1].split('.')[0]+r'\data'
    im = da.from_zarr(fl,component=data)
    im = im[1:]
    im = im.reshape([-1,ncols,im.shape[-2],im.shape[-1]])
    im = im.swapaxes(0,1)
    
    im=im.astype(np.float32)
    im=im*im
    if return_pos is False:
        return im
    else:
        fl_xml = fl.replace('.zarr','.xml')
        x,y = [eval(ln.split('>')[1].split('<')[0]) for ln in open(fl_xml) if 'stage_position' in ln][0]
        return im,x,y
def linear_flat_correction(ims,fl=None,reshape=True,resample=4,vec=[0.1,0.15,0.25,0.5,0.75,0.9]):
    #correct image as (im-bM[1])/bM[0]
    #ims=np.array(ims)
    if reshape:
        ims_pix = np.reshape(ims,[ims.shape[0]*ims.shape[1],ims.shape[2],ims.shape[3]])
    else:
        ims_pix = np.array(ims[::resample])
    ims_pix_sort = np.sort(ims_pix[::resample],axis=0)
    ims_perc = np.array([ims_pix_sort[int(frac*len(ims_pix_sort))] for frac in vec])
    i1,i2=np.array(np.array(ims_perc.shape)[1:]/2,dtype=int)
    #x = ims_perc[:,i1,i2]
    x = np.mean(np.mean(ims_perc,axis=-1),axis=-1)
    X = np.array([x,np.ones(len(x))]).T
    y=ims_perc
    a = np.linalg.inv(np.dot(X.T,X))
    cM = np.swapaxes(np.dot(X.T,np.swapaxes(y,0,-2)),-2,1)
    bM = np.swapaxes(np.dot(a,np.swapaxes(cM,0,-2)),-2,1)
    if fl is not None:
        folder = os.path.dirname(fl)
        if not os.path.exists(folder):
            os.makedirs(folder)
        pickle.dump(bM,open(fl,'wb'))
    return bM
def compose_mosaic(ims,xs_um,ys_um,ims_c=None,um_per_pix=0.108333,rot = 0,return_coords=False):
    dtype = np.float32
    im_ = ims[0]
    szs = im_.shape
    sx,sy = szs[-2],szs[-1]
    ### Apply rotation:
    theta=-np.deg2rad(rot)
    xs_um_ = np.array(xs_um)*np.cos(theta)-np.array(ys_um)*np.sin(theta)
    ys_um_ = np.array(ys_um)*np.cos(theta)+np.array(xs_um)*np.sin(theta)
    ### Calculate per pixel
    xs_pix = np.array(xs_um_)/um_per_pix
    xs_pix = np.array(xs_pix-np.min(xs_pix),dtype=int)
    ys_pix = np.array(ys_um_)/um_per_pix
    ys_pix = np.array(ys_pix-np.min(ys_pix),dtype=int)
    sx_big = np.max(xs_pix)+sx+1
    sy_big = np.max(ys_pix)+sy+1
    dim = [sx_big,sy_big]
    if len(szs)==3:
        dim = [szs[0],sx_big,sy_big]

    if ims_c is None:
        if len(ims)>25:
            try:
                ims_c = linear_flat_correction(ims,fl=None,reshape=False,resample=1,vec=[0.1,0.15,0.25,0.5,0.65,0.75,0.9])
            except:
                imc_c = np.median(ims,axis=0)
        else:
            ims_c = np.median(ims,axis=0)

    im_big = np.zeros(dim,dtype = dtype)
    sh_ = np.nan
    for i,(im_,x_,y_) in enumerate(zip(ims,xs_pix,ys_pix)):
        if ims_c is not None:
            if len(ims_c)==2:
                im_coef,im_inters = np.array(ims_c,dtype = 'float32')
                im__=(np.array(im_,dtype = 'float32')-im_inters)/im_coef
            else:
                ims_c_ = np.array(ims_c,dtype = 'float32')
                im__=np.array(im_,dtype = 'float32')/ims_c_*np.median(ims_c_)
        else:
            im__=np.array(im_,dtype = 'float32')
        im__ = np.array(im__,dtype = dtype)
        im_big[...,x_:x_+sx,y_:y_+sy]=im__
        sh_ = im__.shape
    if return_coords:
        return im_big,xs_pix+sh_[-2]/2,ys_pix+sh_[-1]/2
    return im_big
    
def get_mosaic_image_T(data_fld,resc = 4,icol = 1,frame = 20,force=False,nfls=None):
    save_fld = os.path.dirname(data_fld)+os.sep+'mosaics'
    if not os.path.exists(save_fld): os.makedirs(save_fld)
    fl_save = save_fld+os.sep+os.path.basename(data_fld)+'_col'+str(icol)+'.tiff'
    fl_save = fl_save.replace('.tiff','_annot.png')
    print(fl_save)
    if not os.path.exists(fl_save) or force:
        fls_ = np.sort(glob.glob(data_fld+r'\*.zarr'))
        if nfls is not None:
            fls_=fls_[:nfls]
        ims,xs_um,ys_um=[],[],[]

        for fl in tqdm(fls_[:]):
            im,x,y = read_im(fl,return_pos=True)
            
            if str(frame).lower()=='all':
                ims.append(np.array(np.max(im[icol][:,::resc,::resc],axis=0),dtype=np.float32))
            else:
                ims.append(np.array(im[icol][frame,::resc,::resc],dtype=np.float32))
            xs_um.append(x)
            ys_um.append(y)

        im_big,xs,ys = compose_mosaic([im[::1,::1] for im in ims],xs_um,ys_um,ims_c=None,
                                      um_per_pix=0.1083333*resc,
                                      rot = 0,return_coords= True)
        
        #import tifffile
        #tifffile.imwrite(fl_save,im_big)
        
        
        resc_ = 3
        data_fld = os.path.dirname(fls_[0])
        fig = plt.figure(figsize=(30,30))
        im__ = im_big[::resc_,::resc_]
        #if vmax is None:
        vmax = np.percentile(im__[im__>0],99.5)
        vmin=np.percentile(im__[im__>0],0.5)
        plt.imshow(im_big.T[::resc_,::resc_],vmin=vmin,vmax=vmax,cmap='gray')
        #fig.savefig(fl_save)
        for x_,y_,fl_ in zip(xs,ys,fls_):
            ifov = fl_.split('_')[-1].split('.')[0]
            plt.text(x_/resc_,y_/resc_,ifov,color='r')
        plt.xticks([])
        plt.yticks([])
        fl_save = fl_save.replace('.tiff','_annot.png')
        print(fl_save)
        fig.savefig(fl_save)
        plt.close('all')

In [35]:
data_fld = r'S:\12_04_2025_JenieSample\H8'
get_mosaic_image_T(data_fld,resc = 4,icol = 1,frame = 20,force=True,nfls=None)

S:\12_04_2025_JenieSample\mosaics\H8_col1_annot.png


100%|████████████████████████████████████████████████████████████████████████████████| 563/563 [02:31<00:00,  3.71it/s]


S:\12_04_2025_JenieSample\mosaics\H8_col1_annot.png


In [30]:
data_fld = r'S:\12_04_2025_JenieSample\H2'
get_mosaic_image_T(data_fld,resc = 4,icol = 1,frame = 20,force=False,nfls=None)

S:\12_04_2025_JenieSample\mosaics\H2_col1_annot.png


100%|████████████████████████████████████████████████████████████████████████████████| 563/563 [02:54<00:00,  3.22it/s]


S:\12_04_2025_JenieSample\mosaics\H2_col1_annot.png


In [31]:
data_fld = r'S:\12_04_2025_JenieSample\H3'
get_mosaic_image_T(data_fld,resc = 4,icol = 1,frame = 20,force=False,nfls=None)

S:\12_04_2025_JenieSample\mosaics\H3_col1_annot.png


100%|████████████████████████████████████████████████████████████████████████████████| 563/563 [02:51<00:00,  3.29it/s]


S:\12_04_2025_JenieSample\mosaics\H3_col1_annot.png


In [2]:
for i in [1,2,3,4,5]:
    data_fld = rf'S:\12_04_2025_JenieSample_re\H{i}'
    get_mosaic_image_T(data_fld,resc = 4,icol = 3,frame = 20,force=False,nfls=None)

S:\12_04_2025_JenieSample_re\mosaics\H1_col3_annot.png


100%|████████████████████████████████████████████████████████████████████████████████| 563/563 [02:41<00:00,  3.48it/s]


S:\12_04_2025_JenieSample_re\mosaics\H1_col3_annot.png
S:\12_04_2025_JenieSample_re\mosaics\H2_col3_annot.png


100%|████████████████████████████████████████████████████████████████████████████████| 563/563 [02:36<00:00,  3.61it/s]


S:\12_04_2025_JenieSample_re\mosaics\H2_col3_annot.png
S:\12_04_2025_JenieSample_re\mosaics\H3_col3_annot.png


...

S:\12_04_2025_JenieSample_re\mosaics\H3_col3_annot.png
S:\12_04_2025_JenieSample_re\mosaics\H4_col3_annot.png


100%|████████████████████████████████████████████████████████████████████████████████| 563/563 [02:26<00:00,  3.85it/s]


S:\12_04_2025_JenieSample_re\mosaics\H4_col3_annot.png
S:\12_04_2025_JenieSample_re\mosaics\H5_col3_annot.png


100%|████████████████████████████████████████████████████████████████████████████████| 563/563 [02:15<00:00,  4.15it/s]


S:\12_04_2025_JenieSample_re\mosaics\H5_col3_annot.png
